In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import geopy.distance


In [2]:
df1 = pd.read_excel('bike_pickup_location_v1.xlsx')
df2 = pd.read_excel('/Users/vishu/Documents/playground/Bounce_product_analyst/bike_drop_location_v1 .xlsx')


In [3]:
df1['drop location'] = ''
df2 = df2[['Priority', 'drop hex', 'drop hex  lat', 'drop hex lon','avg_daily_sessions/supply']].dropna().reset_index()
df2 = df2[['Priority', 'drop hex', 'drop hex  lat', 'drop hex lon','avg_daily_sessions/supply']]

In [29]:
df2.head()

,Priority,drop hex,drop hex lat,drop hex lon,avg_daily_sessions/supply,min vehicles,max vehicles,vehicles
0,1.0,8960144b4dbffff,13.084574,77.485690,122.071429,6,8,7
1,2.0,8960144b4c3ffff,13.083513,77.482666,112.857143,2,4,0
2,3.0,89601696927ffff,13.089918,77.483974,110.500000,2,4,0
3,4.0,8960145b547ffff,12.977496,77.570524,90.571429,2,3,3
4,5.0,896016965cbffff,13.078991,77.580048,85.571429,2,3,3


In [6]:
def constraint_1(num):
    if num>= 120:
        return(6,8)
    elif num>=100 and num<120:
        return(2,4)
    elif num>=50 and num<100:
        return(2,3)
    elif num>=30 and num<50:
        return(1,2)
    elif num>=20 and num<30:
        return(1,1)
    elif num>=10 and num<20:
        return(0,1)
    else:
        return(0,0)

In [7]:
df2['min vehicles'] = df2['avg_daily_sessions/supply'].apply(lambda x: int(constraint_1(x)[0]))
df2['max vehicles'] = df2['avg_daily_sessions/supply'].apply(lambda x: int(constraint_1(x)[1]))
df2['vehicles'] = 0
df1['distance'] = 0
df1['drop index'] = ''
skipped_ind = []
ind_array = []

In [8]:
def cal_dist(coords_1, coords_2):
    return geopy.distance.distance(coords_1, coords_2).km


In [9]:

def get_min_ind(dist_array):
    ind = dist_array.index(next(x for x in dist_array if x < 4))
    if df2['vehicles'][ind] < df2['max vehicles'][ind]:
        ind_array.append(ind)
        return ind
    dist_array[ind] = 100
    return get_min_ind(dist_array)

In [10]:
for k in range(0, len(df1)):
    if k in skipped_ind:
        continue
    coords_1 = (df1['lat'][k], df1['lon'][k])
    dist_array = []
    for i in range(0, len(df2)):
        coords_2 = (df2['drop hex  lat'][i], df2['drop hex lon'][i])
        dist = cal_dist(coords_1, coords_2)
        dist_array.append(dist)
    try:
        ind = get_min_ind(dist_array)
    except StopIteration:
        skipped_ind.append(k)
        continue
    
    df1['drop location'][k] = df2['drop hex'][ind]
    df1['distance'][k] = dist_array[ind]
    df1['drop index'][k] = ind
    df2['vehicles'][ind] = df2['vehicles'][ind] + 1



In [40]:
classified_vehicles = df2['vehicles'].sum()
print('total vehicles that were sent to drop location :', classified_vehicles)

total vehicles that were sent to drop location : 322


In [41]:
dist = pd.to_numeric(df1['distance']).sum()
avg_dist = dist/classified_vehicles
print('average distance ', avg_dist)

average distance  2.965377416731192
